UPDATE: Since I couldn't find the correct model that's used for these recomendations, this notebook is incomplete. I did a little exploration which is interesting, but nothing valuable to show. 

# Recommender Systems

There are three kinds of recomendation systems, conent based, collaborative, and popularity. All three of them will be used in this song recomendation program, but first up, is loading this data, along with exploring/understanding it. 

Dataset: 

Song from a bunch of websites, and user ratings. 

Lastfm, secondhand songs, a bunch of ratings by a bunch of users on a bunch of songs. 


Using pandas, we will integrate this data into a single dataframe for further cleaning and modelling.

In [11]:
#import dependencies 

%matplotlib inline

import pandas
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ModuleNotFoundError: No module named 'Recommenders'

In [1]:
#Read userid-songid-listen_count triplets#Read u 
#This step might take time to download data from external sources
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

song_df_1 = pandas.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

#Read song  metadata
song_df_2 =  pandas.read_csv(songs_metadata_file)
 
#Merge the two dataframes above to create input dataframe for recommender systems
song_df = pandas.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

There is an index, song id,listen count, title, etc. What features should we be using? 

Architecture engineering ~ feature engineering

All of the complexity moves away from tuning features, but now to tuning the model, and the hyperparameters used.

We want to predict songs that the user will like, based on the songs that user already likes.

# Next part: data transformation

We are going to combine the song and artist columns together. 

The first 10k songs should be merged into the song title and artists into the same column. 


In [4]:
#subset of the data, and merged.
song_df = song_df.head(10000)

#Merge song title and artist_name columns to make a merged column
song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
#Group songs by the most popular, ie listen count. 
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage']  = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
3660,Sehr kosmisch - Harmonia,45,0.45
4678,Undo - Björk,32,0.32
5105,You're The One - Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.28
3655,Secrets - OneRepublic,28,0.28
4378,The Scientist - Coldplay,27,0.27
4712,Use Somebody - Kings Of Leon,27,0.27
3476,Revelry - Kings Of Leon,26,0.26
1387,Fireflies - Charttraxx Karaoke,24,0.24
1862,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,0.23


Task 1: Count the number of unique users in this dataset.

In [6]:
users = song_df['user_id'].unique()
print("There are " , len(users), " users.")

There are  365  users.


2: What about songs?

In [7]:
songs = song_df['song'].unique()
print("There are " , len(songs), " songs.")

There are  5151  songs.
